In [4]:
import pandas as pd
import numpy as np

#### 13~16

In [23]:
import pandas as pd
import numpy as np

def experiment(sample_size, tau):
    rng = np.random.default_rng()
    # Generate random array x and compute y
    x = rng.uniform(-0.5, 0.5, sample_size)
    x.sort()
    def my_sign(x):
        return np.sign(np.where(x != 0, x, -1))
    y = my_sign(x)

    def flip_labels(y, tau):
        if tau > 0:
            random_vals = np.random.random(size=y.shape)
            flip_indices = random_vals <= tau
            y_flipped = y.copy()
            y_flipped[flip_indices] *= -1
            return y_flipped
        else:
            return y


    # Flip y to -y with probability tau
    y = flip_labels(y, tau)

    # Generate new random array x and compute y
    x_new = rng.uniform(-0.5, 0.5, sample_size)
    x_new.sort()
    y_new = my_sign(x_new)
    y_new = flip_labels(y_new, tau)

    # Define the hypothesis h
    def h(x, s, theta):
        return s * np.sign(x - theta)

    # Compute possible values of theta
    unique_x = np.unique(x)
    diffs = np.diff(unique_x)
    if len(diffs) == 0:
        theta_vals = [-np.inf, np.inf]
    else:
        theta_vals = np.hstack(([-np.inf], unique_x[:-1][diffs != 0] + diffs[diffs != 0] / 2, [np.inf]))

    # Compute Ein(h) for all combinations of s and theta
    misclassified = []
    for s in [-1, 1]:
        for theta in theta_vals:
            predictions = h(x, s, theta)
            misclassified.append((predictions != y).sum())

    # Compute the minimum misclassification error and corresponding s and theta
    min_error_idx = np.argmin(misclassified)
    min_error = misclassified[min_error_idx]
    min_error_s = [-1, 1][min_error_idx // len(theta_vals)]
    min_error_theta = theta_vals[min_error_idx % len(theta_vals)]

    # Return the hypothesis with the minimum Ein as g; break ties by smallest s * theta
    g = h(x, min_error_s, min_error_theta)
    for s in [-1, 1]:
        for theta in theta_vals:
            if s == min_error_s and theta == min_error_theta:
                continue
            error_idx = np.where((s == np.array([-1, 1])))[0][0] * len(theta_vals) + np.abs(theta_vals - theta).argmin()
            if misclassified[error_idx] == min_error and s * theta < min_error_s * min_error_theta:
                g = h(x, s, theta)
                min_error_s = s
                min_error_theta = theta

    Ein_g = min_error / len(x)
    Eout_g = np.mean(h(x_new, min_error_s, min_error_theta) != y_new)
    return Eout_g - Ein_g

##### 13

In [24]:
n_trials = 10000
sample_size = 2
tau = 0   # Set probability of flipping
results = [experiment(sample_size, tau) for i in range(n_trials)]
mean_diff = np.mean(results)

print("Mean of Eout(g, tau) - Ein(g) over", n_trials, "trials:", mean_diff)

<ipython-input-23-9c2b50ed4a7d>:64: RuntimeWarning: invalid value encountered in subtract
  error_idx = np.where((s == np.array([-1, 1])))[0][0] * len(theta_vals) + np.abs(theta_vals - theta).argmin()


Mean of Eout(g, tau) - Ein(g) over 10000 trials: 0.2945


##### 14

In [25]:
n_trials = 10000
sample_size = 128
tau = 0   # Set probability of flipping
results = [experiment(sample_size, tau) for i in range(n_trials)]
mean_diff = np.mean(results)

print("Mean of Eout(g, tau) - Ein(g) over", n_trials, "trials:", mean_diff)

<ipython-input-23-9c2b50ed4a7d>:64: RuntimeWarning: invalid value encountered in subtract
  error_idx = np.where((s == np.array([-1, 1])))[0][0] * len(theta_vals) + np.abs(theta_vals - theta).argmin()


Mean of Eout(g, tau) - Ein(g) over 10000 trials: 0.00369765625


##### 15

In [26]:
n_trials = 10000
sample_size = 2
tau = 0.2   # Set probability of flipping
results = [experiment(sample_size, tau) for i in range(n_trials)]
mean_diff = np.mean(results)

print("Mean of Eout(g, tau) - Ein(g) over", n_trials, "trials:", mean_diff)

<ipython-input-23-9c2b50ed4a7d>:64: RuntimeWarning: invalid value encountered in subtract
  error_idx = np.where((s == np.array([-1, 1])))[0][0] * len(theta_vals) + np.abs(theta_vals - theta).argmin()


Mean of Eout(g, tau) - Ein(g) over 10000 trials: 0.42845


##### 16

In [27]:
n_trials = 10000
sample_size = 128
tau = 0.2   # Set probability of flipping
results = [experiment(sample_size, tau) for i in range(n_trials)]
mean_diff = np.mean(results)

print("Mean of Eout(g, tau) - Ein(g) over", n_trials, "trials:", mean_diff)

<ipython-input-23-9c2b50ed4a7d>:64: RuntimeWarning: invalid value encountered in subtract
  error_idx = np.where((s == np.array([-1, 1])))[0][0] * len(theta_vals) + np.abs(theta_vals - theta).argmin()


Mean of Eout(g, tau) - Ein(g) over 10000 trials: 0.01409140625


#### 17、18

##### best of best algorithm

In [19]:
import pandas as pd
import numpy as np

def experiment1():
    df_train = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)
    X_train = df_train.iloc[:, :-1].to_numpy()
    y_train = df_train.iloc[:, -1].to_numpy()
    sample_size_train = df_train.shape[0]
    num_features = X_train.shape[1]

    df_test = pd.read_csv('ML_HW2_data_test.dat', sep="\t", header=None)
    X_test = df_test.iloc[:, :-1].to_numpy()
    y_test = df_test.iloc[:, -1].to_numpy()
    sample_size_test = df_test.shape[0]

    best_hypothesis = None
    min_error = np.inf

    for i in range(num_features):
        unique_X_train = np.unique(X_train[:, i])
        diffs = np.diff(unique_X_train)
        if len(diffs) == 0:
            theta_vals = [-np.inf, np.inf]
        else:
            theta_vals = np.hstack(([-np.inf], unique_X_train[:-1][diffs != 0] + diffs[diffs != 0] / 2, [np.inf]))

        for s in [-1, 1]:
            for theta in theta_vals:
                predictions = s * np.sign(X_train[:, i] - theta)
                error = np.sum(np.not_equal(predictions, y_train))

                if error < min_error:
                    min_error = error
                    best_hypothesis = (i, s, theta)
                elif error == min_error and i < best_hypothesis[0]:
                    best_hypothesis = (i, s, theta)

    def h(x):
        i, s, theta = best_hypothesis
        return s * np.sign(x[:, i] - theta)

    Ein_g = np.mean(np.not_equal(h(X_train), y_train))
    Eout_g = np.mean(np.not_equal(h(X_test), y_test))

    return Ein_g, Eout_g, Eout_g - Ein_g

Ein_g1, Eout_g1, Eout_Ein_diff1 = experiment1()
print(f"Ein(g1): {Ein_g1:.5f}")
print(f"Eout(g1): {Eout_g1:.5f}")
print(f"Eout(g1) - Ein(g1): {Eout_Ein_diff1:.5f}")


Ein(g1): 0.02604
Eout(g1): 0.07812
Eout(g1) - Ein(g1): 0.05208


#### 19、20

In [21]:
import pandas as pd
import numpy as np

def experiment2():
    df_train = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)
    X_train = df_train.iloc[:, :-1].to_numpy()
    y_train = df_train.iloc[:, -1].to_numpy()
    sample_size_train = df_train.shape[0]
    num_features = X_train.shape[1]

    df_test = pd.read_csv('ML_HW2_data_test.dat', sep="\t", header=None)
    X_test = df_test.iloc[:, :-1].to_numpy()
    y_test = df_test.iloc[:, -1].to_numpy()
    sample_size_test = df_test.shape[0]

    best_hypotheses = [None] * num_features
    min_errors = np.inf * np.ones(num_features)

    for i in range(num_features):
        unique_X_train = np.unique(X_train[:, i])
        diffs = np.diff(unique_X_train)
        if len(diffs) == 0:
            theta_vals = [-np.inf, np.inf]
        else:
            theta_vals = np.hstack(([-np.inf], unique_X_train[:-1][diffs != 0] + diffs[diffs != 0] / 2, [np.inf]))

        for s in [-1, 1]:
            for theta in theta_vals:
                predictions = s * np.sign(X_train[:, i] - theta)
                error = np.mean(np.not_equal(predictions, y_train))

                if error < min_errors[i]:
                    min_errors[i] = error
                    best_hypotheses[i] = (i, s, theta)
                elif error == min_errors[i] and i < best_hypotheses[i][0]:
                    best_hypotheses[i] = (i, s, theta)

    best_hypothesis = best_hypotheses[np.argmax(min_errors)]

    def h(x):
        i, s, theta = best_hypothesis
        return s * np.sign(x[:, i] - theta)

    Ein_g2 = np.mean(np.not_equal(h(X_train), y_train))
    Eout_g2 = np.mean(np.not_equal(h(X_test), y_test))

    return Ein_g2, Eout_g2, Eout_g2 - Ein_g2

Ein_g2, Eout_g2, Eout_Ein_diff2 = experiment2()
print(f"Ein(g2): {Ein_g2:.5f}")
print(f"Eout(g2): {Eout_g2:.5f}")
print(f"Eout(g2) - Ein(g2): {Eout_Ein_diff2:.5f}")


Ein(g2): 0.32812
Eout(g2): 0.42188
Eout(g2) - Ein(g2): 0.09375


In [22]:
print(f"delta_Ein = {Ein_g2 - Ein_g1:.5f}")
print(f"delta_Eout = {Eout_g2 - Eout_g1:.5f}")

delta_Ein = 0.30208
delta_Eout = 0.34375
